<a href="https://colab.research.google.com/github/sayeem038/Character-level-recurrent-neural-network-RNN-to-generate-handwritten-like-text./blob/main/HANDWRITTEN_TEXT_GENERATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Set Up Your Colab Environment
Open Google Colab: Go to Google Colab.

Create a New Notebook: Click on "File" > "New notebook".

Step 2: Import Necessary Libraries
Start by importing the necessary libraries.

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
import os


Step 3: Load and Preprocess the Dataset
Download the Dataset:
Download the IAM Handwriting Database from Papers with Code.

Upload the Dataset to Colab:

Click on the folder icon on the left sidebar.
Click on the upload icon and upload your dataset files.
Load the Data:
Assuming the dataset is in a text file format:

In [7]:
# Path to your uploaded file
path_to_file = "/content/IAM-0000003255-109cf78d.jpg"

# Load the image using an image processing library like Pillow
from PIL import Image
image = Image.open(path_to_file)

# Further processing of the image as needed
# ...

Step 4: Define the Model
Define an RNN model using LSTM.

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Sample vocabulary for demonstration purposes
vocab = ["a", "b", "c", "d", "e"]  # Example vocabulary
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

# Define BATCH_SIZE
BATCH_SIZE = 64

model = Sequential([
    Embedding(vocab_size, embedding_dim, batch_input_shape=[BATCH_SIZE, None]),
    LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    Dense(vocab_size)
])

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

# Print model summary
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           1280      
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 5)             5125      
                                                                 
Total params: 5253381 (20.04 MB)
Trainable params: 5253381 (20.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Step 5: Train the Model
Train the model and save checkpoints.

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import os

# Sample vocabulary for demonstration purposes
vocab = ["a", "b", "c", "d", "e"]  # Example vocabulary
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

# Define BATCH_SIZE
BATCH_SIZE = 64

model = Sequential([
    Embedding(vocab_size, embedding_dim, batch_input_shape=[BATCH_SIZE, None]),
    LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    Dense(vocab_size)
])

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

# Print model summary
model.summary()

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

# Create a sample dataset (replace this with your actual data loading)
# Example: Generating random data for demonstration
import numpy as np
x_train = np.random.randint(0, vocab_size, (BATCH_SIZE, 10))  # Ensure batch size is BATCH_SIZE
y_train = np.random.randint(0, vocab_size, (BATCH_SIZE, 10))

# Train the model
EPOCHS = 10
history = model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[checkpoint_callback])


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (64, None, 256)           1280      
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_2 (Dense)             (64, None, 5)             5125      
                                                                 
Total params: 5253381 (20.04 MB)
Trainable params: 5253381 (20.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 1.6096
Epoch 2/10
1/1 [==============================] - 1s 1s/step - loss: 1.6030
Epoch 3/10
1/1 [==============================] - 2s 2s/step - loss: 1.6002
Epoch 4/10
1/1 [==============================] - 

Step 6: Generate Text
Load the trained model and generate new text.

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import numpy as np
import os

# Sample vocabulary for demonstration purposes
vocab = ["a", "b", "c", "d", "e"]  # Example vocabulary
vocab_size = len(vocab)

# Character to index and index to character mappings
char2idx = {char: idx for idx, char in enumerate(vocab)}
idx2char = np.array(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'

# Rebuild the model for text generation with batch_size=1
# Use the same vocabulary size as the training model
model = Sequential([
    Embedding(vocab_size, embedding_dim, batch_input_shape=[1, None]),
    LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    Dense(vocab_size)
])

# Load the latest checkpoint if available
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
if latest_checkpoint:
    model.load_weights(latest_checkpoint)
else:
    print("No checkpoint found. Please ensure that the checkpoint directory contains valid checkpoints.")

model.build(tf.TensorShape([1, None]))

def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures result in more predictable text.
    # Higher temperatures result in more surprising text.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # Remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # Using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # We pass the predicted character as the next input to the model
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

# Use a start string that only contains characters from your vocabulary
print(generate_text(model, start_string="a"))  # Ensure the start string is in the vocab


adcbbdaedeeaedccedcdbdcaaabaaabebeaccdbbdadcddccabbadabbbecacdeaceeccbddaebbedcabdaabdeabbabdccdabddbedacdcdceabccbadbdccbdedbcaddeedecbebcddbdbdadbdaaeebdebbbeeeeeecedeceeaebabbbcdeabbcaabbacdacbebddeeadcbaeaeadaddaedacaeeacddeccbbbeacdaebddbeaedcdabebeeeeedeaeddebdbaeeadecabbddbdadcddaadbadbbbbdaaabeacbeaaddabdbbdabdbdcaaddabedcbbdcaabdbaaecabebbcaaaccedebcebcabedaeccbadabecaeecbeecececbaacddbcadddeddacaadbbabbcbadbbaeabdeeaeceeeadecbbaeeaeedaaedbbbdcbdaababaabeabaedccdbdcbdddccebaedbaeecabaabcbaadadabdaedcbceeabdabdddbdbdacdedcabecbeabdeaacbbaacbaddbbcaadaecadaabaaeebbcecbddeaeadcddbbdddbaeadbbaedbeeeeadeaeecdaaebbaeacbddccbeaddabdcabbaaadbbcbaababbddecdbdabacbbadbcaadacaaeaeadcadebddeeededabdaeabdaaebdbbbbeeabbeedccedddabaadbadbcaaaebaceaabdaabbdaabddcdbadbcadeabebedbeeccdbeaddaddbabddaadaabdeeeeeceeeecddaaddddabbbcbaeeacbdeeeecbebaeedbebbeeacceabcccecacaeaededcaddbaccabbbacabdbddeddbcbedbaaecaecbadedcdabeacaeadeaeadddbdddbacebbcecebecedeadccbbadddaacacadacadbdeaddcaddddcabdedddbbb